# 課題（ilect上で提出してください）

## Matplotlibを使ったデータ可視化

下記の「#common」で始まるセルの中で指定されたリンク先にある購買データをベースに以下の問いに答えて、課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
購買者（CustomerID）の各合計購買金額を算出し、さらに金額をベースに降順に並び替えをします。カラムがCustomerIDと合計金額のあるテーブルを作成してください。そこから、購買者を10等分にグループ分けします（例：100人いたら、10人ずつにグループ分けします。）。それぞれのグループでの合計購買金額の範囲と、それぞれの金額合計値を算出してください（このアプローチをデシル分析といいます。）。この結果を用いて、パレートの法則（上位2割の顧客が売上全体の8割を占める）を確かめるため、**それぞれのグループが売上の何割を占めるか（Pandas.Series形式で売上が小さいグループが先頭、大きいほど後になるようにしてください）**提出してください。
(なお、マーケティング戦略では、このように顧客を分けることをセグメンテーションといい、上位2割に絞ってアプローチを仕掛けることをターゲティングといいます。それぞれの戦略によりますが、優良顧客に的を絞った方が投資対効果が高いことが多いため、このようなアプローチを取ることがあります。)

提出するときは、以下の点に注意してください。  
>- 以下の関数homeworkの!!WRITE ME!!に処理を書いてください。(**「!!WRITE ME!!」は消して、記入してください。**)
>- 実際の提出は記述されたhomework関数全てになり、**提出場所はilect右上にある青字のHomeworkドロップダウンメニューからSaveしてください。Solutionに関数を貼り付けてからSaveを押してください。**
>- 前処理は以下のコードを使ってください。  
>- 返り値がPandas.Series形式になるようにしてください。  
>- 解答は、my_result=pd.Series([0,0.01,0.02,0.021,0.022,0.025,0.025,0.1,0.2,0.6]) のような形になります。(売上割合は昇順にして、合計は1になるようにします。)

https://note.nkmk.me/python-pandas-assign-append/

In [0]:
#Gdriveにマウント
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd 

In [0]:
# common
import pandas as pd
import numpy as np

## init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [0]:
# working place. everything 
def homework1(target_online_retail_data_tb):
    #TotalPriceで降順にソートしたデータフレームを新しい変数に入れる
    target_online_retail_data_tb_sort = target_online_retail_data_tb.sort_values('TotalPrice', ascending=False)
    dfD = target_online_retail_data_tb_sort
    #新しいカラム'Decile'を作り、値を０で埋める
    dfD['Decile']=0
    #グループ分けするラベルのリストを作成
    labels=["decile01","decile02","decile03","decile04","decile05","decile06","decile07","decile08","decile09"]
    #グルーピングのサイズの計算
    decilN=len(target_online_retail_data_tb)//10
    #グルーピング
    for i in range(1,10):
      dfD['Decile'][(i-1)*decilN:i*decilN]=labels[i-1]
    #最後のグループは別の処理を使う
    dfD['Decile'][9*decilN:]="decile10"
    #各グループの最小金額と最大金額
    min = dfD.groupby('Decile').min()
    max = dfD.groupby('Decile').max()
    #各グループの合計金額
    dfDs = dfD.groupby('Decile').sum()
    dfDs2 = dfDs.sort_values(by=["TotalPrice"], ascending=True) #TotalPriceについて降順(大きい順)で並び替え
    dfDs2['share']=0.00
    ATP=dfD['TotalPrice'].sum()
    for i in range(10):
      dfDs2['share'][i]=dfDs2['TotalPrice'][i]/ATP*100
    my_result=pd.Series(dfDs2['share'].values)
    return my_result, dfDs2 

In [27]:
#1.購買者（CustomerID）の各合計購買金額を算出し、さらに金額をベースに降順に並び替えをします。
#カラムがCustomerIDと合計金額のあるテーブルを作成してください。
dfCT=target_online_retail_data_tb.groupby('CustomerID', as_index=False)['TotalPrice'].sum() 
dfCTA=dfCT.sort_values('TotalPrice', ascending=False)
#TotalPriceで絞らないと数値データ全体でsumする
#
dfCTA

,CustomerID,TotalPrice
1690,14646.0,280206.02
4202,18102.0,259657.30
3729,17450.0,194550.79
3009,16446.0,168472.50
1880,14911.0,143825.06
...,...,...
4099,17956.0,12.75
3015,16454.0,6.90
1794,14792.0,6.20
3218,16738.0,3.75


In [28]:
#そこから、購買者を10等分にグループ分けします（例：100人いたら、10人ずつにグループ分けします。）。
bins=10
dfCTA['Dgroup']=pd.qcut(dfCTA['CustomerID'] ,bins)
dfCTA

,CustomerID,TotalPrice,Dgroup
1690,14646.0,280206.02,"(14110.4, 14708.2]"
4202,18102.0,259657.30,"(17686.8, 18287.0]"
3729,17450.0,194550.79,"(17080.4, 17686.8]"
3009,16446.0,168472.50,"(15882.8, 16481.2]"
1880,14911.0,143825.06,"(14708.2, 15299.0]"
...,...,...,...
4099,17956.0,12.75,"(17686.8, 18287.0]"
3015,16454.0,6.90,"(15882.8, 16481.2]"
1794,14792.0,6.20,"(14708.2, 15299.0]"
3218,16738.0,3.75,"(16481.2, 17080.4]"


In [29]:
#それぞれのグループでの合計購買金額の範囲と、それぞれの金額合計値を算出してください（このアプローチをデシル分析といいます。）。
dfDT=dfCTA.groupby('Dgroup', as_index=False)['TotalPrice'].agg(["min","max","sum"])
dfDT

,min,max,sum
Dgroup,,,
"(12345.999, 12905.6]",38.10,124914.53,1159983.940
"(12905.6, 13505.6]",0.00,58825.83,837089.381
"(13505.6, 14110.4]",45.60,65164.79,847862.601
"(14110.4, 14708.2]",35.40,280206.02,1068967.110
"(14708.2, 15299.0]",6.20,143825.06,883173.340
"(15299.0, 15882.8]",15.00,60767.90,744079.491
"(15882.8, 16481.2]",6.90,168472.50,865242.741
"(16481.2, 17080.4]",3.75,66653.56,661561.440
"(17080.4, 17686.8]",25.50,194550.79,951904.340


In [26]:
#この結果を用いて、パレートの法則（上位2割の顧客が売上全体の8割を占める）を確かめるため、それぞれのグループが売上の何割を占めるか（Pandas.Series形式で売上が小さいグループが先頭、大きいほど後になるようにしてください）提出してください。
dfDT['share']=dfDT['sum']/dfDT['sum'].sum()
dfDT.sort_values('share',ascending=False)

,min,max,sum,share
Dgroup,,,,
"(12345.999, 12905.6]",38.10,124914.53,1159983.940,0.130168
"(14110.4, 14708.2]",35.40,280206.02,1068967.110,0.119955
"(17080.4, 17686.8]",25.50,194550.79,951904.340,0.106819
"(17686.8, 18287.0]",12.75,259657.30,891543.520,0.100045
"(14708.2, 15299.0]",6.20,143825.06,883173.340,0.099106
"(15882.8, 16481.2]",6.90,168472.50,865242.741,0.097094
"(13505.6, 14110.4]",45.60,65164.79,847862.601,0.095144
"(12905.6, 13505.6]",0.00,58825.83,837089.381,0.093935
"(15299.0, 15882.8]",15.00,60767.90,744079.491,0.083497


In [0]:
#1)購買者（CustomerID）の各合計購買金額を算出し、
df1=target_online_retail_data_tb.groupby('CustomerID' ,as_index=False)['TotalPrice'].sum()
df1

,CustomerID,TotalPrice
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
...,...,...
4334,18280.0,180.60
4335,18281.0,80.82
4336,18282.0,178.05
4337,18283.0,2094.88


In [0]:
#2)さらに金額をベースに降順に並び替えをします。
#3)カラムがCustomerIDと合計金額のあるテーブルを作成してください。
df1 = df1.sort_values('TotalPrice',ascending=False)
df1

,CustomerID,TotalPrice
1690,14646.0,280206.02
4202,18102.0,259657.30
3729,17450.0,194550.79
3009,16446.0,168472.50
1880,14911.0,143825.06
...,...,...
4099,17956.0,12.75
3015,16454.0,6.90
1794,14792.0,6.20
3218,16738.0,3.75


In [0]:
#4)そこから、購買者を10等分にグループ分けします（例：100人いたら、10人ずつにグループ分けします。）。
df1['label'] =pd.qcut(df1['TotalPrice'], 10, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10'])
df1

,CustomerID,TotalPrice,label
1690,14646.0,280206.02,Q10
4202,18102.0,259657.30,Q10
3729,17450.0,194550.79,Q10
3009,16446.0,168472.50,Q10
1880,14911.0,143825.06,Q10
...,...,...,...
4099,17956.0,12.75,Q1
3015,16454.0,6.90,Q1
1794,14792.0,6.20,Q1
3218,16738.0,3.75,Q1


In [0]:
#5)それぞれのグループでの合計購買金額の範囲と、
#6)それぞれの金額合計値を算出してください（このアプローチをデシル分析といいます。）。
df2 = df1.groupby('label').agg(["count", "mean", "std","min","max","sum"])['TotalPrice']
df2

,count,mean,std,min,max,sum
label,,,,,,
Q1,434,105.862120,35.862280,0.00,156.51,45944.160
Q2,434,200.796037,26.997567,156.58,250.04,87145.480
Q3,434,304.419055,28.030377,250.15,351.65,132117.870
Q4,434,414.419242,39.654781,351.67,489.60,179857.951
Q5,434,582.722306,53.173772,490.22,674.45,252901.481
Q6,433,793.977855,78.230640,674.52,940.67,343792.411
Q7,434,1133.073065,120.334056,942.26,1353.74,491753.710
Q8,434,1678.473226,202.536703,1353.74,2056.89,728457.380
Q9,434,2719.020740,452.934273,2059.45,3645.43,1180055.001


In [0]:
#7)**それぞれのグループが売上の何割を占めるか
TotalSum=target_online_retail_data_tb.sum()['TotalPrice']
df2['Per']= df2['sum'] / TotalSum
df2

KeyboardInterrupt: ignored

In [0]:






    #8)（Pandas.Series形式で売上が小さいグループが先頭、大きいほど後になるようにしてください）
    my_result = pd.Series(df2['Per'])

    #9)**提出してください。
    return my_result

In [0]:
target_online_retail_data_tb

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,5,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,5,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60


In [0]:
#CustomerID毎の合計金額を出す
dfCus = target_online_retail_data_tb.groupby('CustomerID')
list1=dfCus.sum()['TotalPrice'].values
index1 =dfCus.sum().index
columns1 =["TotalPrice"]
dfN=pd.DataFrame(data=list1, index=index1, columns=columns1)
dfN

,TotalPrice
CustomerID,
12346.0,77183.60
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12350.0,334.40
...,...
18280.0,180.60
18281.0,80.82
18282.0,178.05


In [0]:
#CustomerIDでまとめる
dfCus = target_online_retail_data_tb.groupby('CustomerID')
dfCus.size()

CustomerID
12346.0      1
12347.0    182
12348.0     31
12349.0     73
12350.0     17
          ... 
18280.0     10
18281.0      7
18282.0     12
18283.0    756
18287.0     70
Length: 4339, dtype: int64

In [0]:
#CustomaerID毎に合計金額を出す
dfCus.sum()

,Quantity,UnitPrice,TotalPrice
CustomerID,,,
12346.0,74215,1.04,77183.60
12347.0,2458,481.21,4310.00
12348.0,2341,178.71,1797.24
12349.0,631,605.10,1757.55
12350.0,197,65.30,334.40
...,...,...,...
18280.0,45,47.65,180.60
18281.0,54,39.36,80.82
18282.0,103,62.39,178.05


In [0]:
dfCus.sum().index

Float64Index([12346.0, 12347.0, 12348.0, 12349.0, 12350.0, 12352.0, 12353.0,
              12354.0, 12355.0, 12356.0,
              ...
              18273.0, 18274.0, 18276.0, 18277.0, 18278.0, 18280.0, 18281.0,
              18282.0, 18283.0, 18287.0],
             dtype='float64', name='CustomerID', length=4339)

In [0]:
#CustomerIDとTotalPriceだけのDFを作る
dfCT=dfCus.sum()['TotalPrice'].values
dfCT

array([77183.6 ,  4310.  ,  1797.24, ...,   178.05,  2094.88,  1837.28])

In [0]:
target_online_retail_data_tb_sort = target_online_retail_data_tb.sort_values('TotalPrice', ascending=False)
target_online_retail_data_tb_sort

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,5,168469.60
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,5,77183.60
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,5,38970.00
173382,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,5,8142.75
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,5,7144.72
...,...,...,...,...,...,...,...,...,...,...
279324,561284,22167,OVAL WALL MIRROR DIAMANTE,1,2011-07-26 12:24:00,0.00,16818.0,United Kingdom,5,0.00
285657,561916,M,Manual,1,2011-08-01 11:44:00,0.00,15581.0,United Kingdom,5,0.00
395529,571035,M,Manual,1,2011-10-13 12:50:00,0.00,12446.0,RSA,5,0.00
439361,574469,22385,JUMBO BAG SPACEBOY DESIGN,12,2011-11-04 11:55:00,0.00,12431.0,Australia,5,0.00


In [0]:
import pandas as pd
import numpy as np
#DFを作る
list1=dfCus.sum()['TotalPrice'].values
index1 =dfCus.sum().index
columns1 =["TotalPrice"]
dfN=pd.DataFrame(data=list1, index=index1, columns=columns1)
dfN

,TotalPrice
CustomerID,
12346.0,77183.60
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12350.0,334.40
...,...
18280.0,180.60
18281.0,80.82
18282.0,178.05


In [0]:
#エクセルで保存
path = "/content/gdrive/My Drive/Colab Notebooks/GCI2019/week3/"

target_online_retail_data_tb.to_excel(path + 'homework3.xlsx')

In [0]:
ls

homework3.xlsx  sample_data/


In [0]:
my_result, dfDs2 = homework1(target_online_retail_data_tb)
print(my_result)
print(dfDs2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0     0.539001
1     1.229035
2     2.041860
3     3.219462
4     4.525899
5     6.193816
6     7.197534
7     8.522584
8    12.133614
9    54.397195
dtype: float64
          Quantity   UnitPrice   CustomerID    TotalPrice      share
Decile                                                              
decile10     73653   39169.914  628100172.0  4.803261e+04   0.539001
decile09     99805   70435.180  626511925.0  1.095244e+05   1.229035
decile08    193534   87144.270  619680507.0  1.819585e+05   2.041860
decile07    225885  117988.230  617781014.0  2.868994e+05   3.219462
decile06    461897   86272.040  606205704.0  4.033213e+05   4.525899
decile05    369538  132532.690  600635622.0  5.519562e+05   6.193816
decile04    313193  155079.750  597439549.0  6.414016e+05   7.197534
decile03    417499  127286.150  595178324.0  7.594822e+05   8.522584
decile02    556375  148987.760  598601228.0  1.081276e+06  12.133614
decile01   2470317  275104.630  595841025.0  4.847556e+06  54.397195


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
# working place. everything 
def homework2(target_online_retail_data_tb):
    #CustomerID毎の合計金額を出す
    dfCus = target_online_retail_data_tb.groupby('CustomerID')
    list1=dfCus.sum()['TotalPrice'].values
    index1 =dfCus.sum().index
    columns1 =["TotalPrice"]
    dfN=pd.DataFrame(data=list1, index=index1, columns=columns1)
    #グルーピング
    #deciles=["decile01","decile02","decile03","decile04","decile05","decile06","decile07","decile08","decile09","decile10"]
    #bins=10
    dfN['decile'] = pd.qcut(dfN['TotalPrice'], 10, labels=["decile01","decile02","decile03","decile04","decile05","decile06","decile07","decile08","decile09","decile10"])
    #グループごとに購入金額の合計をとる
    dfP = dfN.groupby('decile').sum()
    #グループ毎の割合を出す
    dfP['share']=dfP['TotalPrice']/dfN['TotalPrice'].sum()
    #my_result=pd.Series(dfA.values)
    my_result = dfP['share']
    return my_result

In [0]:
my_result = homework2(target_online_retail_data_tb)
my_result

decile
decile01    0.005156
decile02    0.009779
decile03    0.014826
decile04    0.020183
decile05    0.028380
decile06    0.038579
decile07    0.055182
decile08    0.081744
decile09    0.132421
decile10    0.613751
Name: share, dtype: float64

In [0]:
#deciles=["decile01","decile02","decile03","decile04","decile05","decile06","decile07","decile08","decile09","decile10"]
#bins=10
dfN['decile'] = pd.qcut(dfN['TotalPrice'], 10, labels=["decile01","decile02","decile03","decile04","decile05","decile06","decile07","decile08","decile09","decile10"])

In [0]:
#グループごとに購入金額の合計をとる
dfP = dfN.groupby('decile').sum()

In [0]:
dfP

,TotalPrice
decile,
decile01,45944.160
decile02,87145.480
decile03,132117.870
decile04,179857.951
decile05,252901.481
decile06,343792.411
decile07,491753.710
decile08,728457.380
decile09,1180055.001


In [0]:
dfP['TotalPrice']

decile
decile01      45944.160
decile02      87145.480
decile03     132117.870
decile04     179857.951
decile05     252901.481
decile06     343792.411
decile07     491753.710
decile08     728457.380
decile09    1180055.001
decile10    5469382.460
Name: TotalPrice, dtype: float64

In [0]:
dfN['TotalPrice'].sum()

8911407.904

In [0]:
dfP['TotalPrice']/dfN['TotalPrice'].sum()

decile
decile01    0.005156
decile02    0.009779
decile03    0.014826
decile04    0.020183
decile05    0.028380
decile06    0.038579
decile07    0.055182
decile08    0.081744
decile09    0.132421
decile10    0.613751
Name: TotalPrice, dtype: float64

In [0]:
#グループ毎の割合を出す
dfP['share']=dfP['TotalPrice']/dfN['TotalPrice'].sum()
dfP

,TotalPrice,share
decile,,
decile01,45944.160,0.005156
decile02,87145.480,0.009779
decile03,132117.870,0.014826
decile04,179857.951,0.020183
decile05,252901.481,0.028380
decile06,343792.411,0.038579
decile07,491753.710,0.055182
decile08,728457.380,0.081744
decile09,1180055.001,0.132421


In [0]:
#TotalPriceで降順にソートしたデータフレームを新しい変数に入れる
target_online_retail_data_tb_sort = target_online_retail_data_tb.sort_values('TotalPrice', ascending=False)
dfD = target_online_retail_data_tb_sort
print("データ数：", len(dfD))
#グルーピング
dfD['decile'] = pd.qcut(dfD['TotalPrice'], 10, labels=["decile01","decile02","decile03","decile04","decile05","decile06","decile07","decile08","decile09","decile10"])
dfD

データ数： 397924


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice,decile
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,5,168469.60,decile10
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,5,77183.60,decile10
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,5,38970.00,decile10
173382,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,5,8142.75,decile10
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,5,7144.72,decile10
...,...,...,...,...,...,...,...,...,...,...,...
279324,561284,22167,OVAL WALL MIRROR DIAMANTE,1,2011-07-26 12:24:00,0.00,16818.0,United Kingdom,5,0.00,decile01
285657,561916,M,Manual,1,2011-08-01 11:44:00,0.00,15581.0,United Kingdom,5,0.00,decile01
395529,571035,M,Manual,1,2011-10-13 12:50:00,0.00,12446.0,RSA,5,0.00,decile01
439361,574469,22385,JUMBO BAG SPACEBOY DESIGN,12,2011-11-04 11:55:00,0.00,12431.0,Australia,5,0.00,decile01


In [0]:
#グループごとに購入金額の合計をとる
dfP = dfD.groupby('decile').sum()
dfP['TotalPrice']

decile
decile01    4.939566e+04
decile02    1.265776e+05
decile03    1.667496e+05
decile04    2.903730e+05
decile05    3.983867e+05
decile06    6.625748e+05
decile07    5.638540e+05
decile08    7.913080e+05
decile09    1.050882e+06
decile10    4.811306e+06
Name: TotalPrice, dtype: float64

In [0]:
dfP

,Quantity,UnitPrice,CustomerID,TotalPrice,share
decile,,,,,
decile01,74352,40532.964,639310149.0,4.939566e+04,0.0
decile02,107957,82589.380,692892758.0,1.265776e+05,0.0
decile03,187439,74389.430,550737314.0,1.667496e+05,0.0
decile04,227516,120261.170,621169555.0,2.903730e+05,0.0
decile05,458102,83673.290,596505469.0,3.983867e+05,0.0
decile06,440861,148024.590,711099500.0,6.625748e+05,0.0
decile07,265482,142048.060,514376536.0,5.638540e+05,0.0
decile08,426325,131136.730,613583019.0,7.913080e+05,0.0
decile09,535633,145261.170,565759733.0,1.050882e+06,0.0


In [0]:
type(dfP)

pandas.core.frame.DataFrame

In [0]:
dfD['TotalPrice'].sum()

8911407.904000001

In [0]:
dfP['share']=dfP['TotalPrice']/dfD['TotalPrice'].sum()
dfP

,Quantity,UnitPrice,CustomerID,TotalPrice,share
decile,,,,,
decile01,74352,40532.964,639310149.0,4.939566e+04,0.005543
decile02,107957,82589.380,692892758.0,1.265776e+05,0.014204
decile03,187439,74389.430,550737314.0,1.667496e+05,0.018712
decile04,227516,120261.170,621169555.0,2.903730e+05,0.032584
decile05,458102,83673.290,596505469.0,3.983867e+05,0.044705
decile06,440861,148024.590,711099500.0,6.625748e+05,0.074351
decile07,265482,142048.060,514376536.0,5.638540e+05,0.063273
decile08,426325,131136.730,613583019.0,7.913080e+05,0.088797
decile09,535633,145261.170,565759733.0,1.050882e+06,0.117925


In [0]:
dfP['share']

decile
decile01    0.005543
decile02    0.014204
decile03    0.018712
decile04    0.032584
decile05    0.044705
decile06    0.074351
decile07    0.063273
decile08    0.088797
decile09    0.117925
decile10    0.539904
Name: share, dtype: float64

In [0]:
srA=pd.Series(dfA.values)
srA

0    0.005543
1    0.014204
2    0.018712
3    0.032584
4    0.044705
5    0.074351
6    0.063273
7    0.088797
8    0.117925
9    0.539904
dtype: float64

In [0]:
from pandas import Series, DataFrame

In [0]:
df = pd.DataFrame(decile)
df

,TotalPrice
540421,"(35.4, 168469.6]"
61619,"(35.4, 168469.6]"
222680,"(35.4, 168469.6]"
173382,"(35.4, 168469.6]"
348325,"(35.4, 168469.6]"
...,...
279324,"(-0.001, 1.95]"
285657,"(-0.001, 1.95]"
395529,"(-0.001, 1.95]"
439361,"(-0.001, 1.95]"


In [0]:
groupingP=df.groupby('TotalPrice')
groupingP.size()

TotalPrice
(-0.001, 1.95]      40495
(1.95, 3.75]        44004
(3.75, 5.8]         35434
(5.8, 8.5]          40025
(8.5, 11.8]         39154
(11.8, 15.0]        47135
(15.0, 17.4]        34302
(17.4, 20.8]        40995
(20.8, 35.4]        37612
(35.4, 168469.6]    38768
dtype: int64

In [0]:
type(grouping.groups)

dict

In [0]:
dic=grouping.groups

In [0]:
dic

{Interval(-0.001, 1.95, closed='right'): Int64Index([ 98264, 285000, 139232,  30469,  37714, 133280, 395608, 487745,
             500419, 113957,
             ...
             139453,   9302, 361825, 480649, 130188, 279324, 285657, 395529,
             439361,  47070],
            dtype='int64', length=40495),
 Interval(1.95, 3.75, closed='right'): Int64Index([280513, 191639, 207140, 338489, 118069,    637, 207218, 510911,
             492700,  58243,
             ...
             203929, 524958, 409035, 100928, 526079, 216399, 208560, 203928,
             356791, 216402],
            dtype='int64', length=44004),
 Interval(3.75, 5.8, closed='right'): Int64Index([407974, 348329, 405125, 454415, 351170, 222214, 538930,  70933,
             107197, 386186,
             ...
             532873, 426173, 461826, 216314, 461245, 426091, 359565, 143933,
             420968, 474858],
            dtype='int64', length=35434),
 Interval(5.8, 8.5, closed='right'): Int64Index([434409, 284551, 5310

In [0]:
len(groupingP.groups)

10

In [0]:
#groupbyでグループ分けしたものに名前を付けて新しいカラムに挿入したい#df1.groupby(['region', 'city'], as_index = False)['birth_year'].mean()
df['decile']=

In [0]:
decile.index.to_list()

In [0]:
decile.value_counts

<bound method IndexOpsMixin.value_counts of 540421    (35.4, 168469.6]
61619     (35.4, 168469.6]
222680    (35.4, 168469.6]
173382    (35.4, 168469.6]
348325    (35.4, 168469.6]
                ...       
279324      (-0.001, 1.95]
285657      (-0.001, 1.95]
395529      (-0.001, 1.95]
439361      (-0.001, 1.95]
47070       (-0.001, 1.95]
Name: TotalPrice, Length: 397924, dtype: category
Categories (10, interval[float64]): [(-0.001, 1.95] < (1.95, 3.75] < (3.75, 5.8] < (5.8, 8.5] < ... <
                                     (15.0, 17.4] < (17.4, 20.8] < (20.8, 35.4] < (35.4, 168469.6]]>

In [0]:
type(homework(target_online_retail_data_tb))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


pandas.core.series.Series

In [0]:
#購買者（CustomerID）の各合計購買金額を算出し、さらに金額をベースに降順に並び替えをします。
target_online_retail_data_tb_sort = target_online_retail_data_tb.sort_values('TotalPrice', ascending=False)
target_online_retail_data_tb_sort

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,5,168469.60
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,5,77183.60
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,5,38970.00
173382,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,5,8142.75
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,5,7144.72
...,...,...,...,...,...,...,...,...,...,...
279324,561284,22167,OVAL WALL MIRROR DIAMANTE,1,2011-07-26 12:24:00,0.00,16818.0,United Kingdom,5,0.00
285657,561916,M,Manual,1,2011-08-01 11:44:00,0.00,15581.0,United Kingdom,5,0.00
395529,571035,M,Manual,1,2011-10-13 12:50:00,0.00,12446.0,RSA,5,0.00
439361,574469,22385,JUMBO BAG SPACEBOY DESIGN,12,2011-11-04 11:55:00,0.00,12431.0,Australia,5,0.00


In [0]:
#まず分けてやってみる
#使用するデータ数
print(len(target_online_retail_data_tb))
#データを10分割：この場合９つまで均等で最後の一つはちょっと少ない＝＞デシル分析がどのように行われているか？
decilN=len(target_online_retail_data_tb)//10
print("分割：decilN=",decilN)
decil10=len(target_online_retail_data_tb)-decilN*9
print("最後：decil10=",decil10)
print(decilN*9+decil10)

397924
分割：decilN= 39792
最後：decil10= 39796
397924


In [0]:
#元になるデータをdfDに入れる
dfD=target_online_retail_data_tb_sort
dfD

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,5,168469.60
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,5,77183.60
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,5,38970.00
173382,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,5,8142.75
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,5,7144.72
...,...,...,...,...,...,...,...,...,...,...
279324,561284,22167,OVAL WALL MIRROR DIAMANTE,1,2011-07-26 12:24:00,0.00,16818.0,United Kingdom,5,0.00
285657,561916,M,Manual,1,2011-08-01 11:44:00,0.00,15581.0,United Kingdom,5,0.00
395529,571035,M,Manual,1,2011-10-13 12:50:00,0.00,12446.0,RSA,5,0.00
439361,574469,22385,JUMBO BAG SPACEBOY DESIGN,12,2011-11-04 11:55:00,0.00,12431.0,Australia,5,0.00


In [0]:
dfD[0:10] #0～9までの10個を抽出

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice,Deli
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,5,168469.60,decile01
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,5,77183.60,decile01
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,5,38970.00,decile01
173382,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,5,8142.75,decile01
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,5,7144.72,decile01
160546,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-04-18 13:20:00,2.10,15749.0,United Kingdom,5,6539.40,decile01
52711,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-01-11 12:55:00,2.10,15749.0,United Kingdom,5,6539.40,decile01
421601,573003,23084,RABBIT NIGHT LIGHT,2400,2011-10-27 12:11:00,2.08,14646.0,Netherlands,5,4992.00,decile01
52709,540815,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,2011-01-11 12:55:00,2.55,15749.0,United Kingdom,5,4921.50,decile01
160542,550461,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,2011-04-18 13:20:00,2.40,15749.0,United Kingdom,5,4632.00,decile01


In [0]:
dfD[10:20]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice,Deli
52772,540818,48185,DOORMAT FAIRY CAKE,670,2011-01-11 12:57:00,6.75,15749.0,United Kingdom,5,4522.50,decile01
244419,558526,23173,REGENCY TEAPOT ROSES,540,2011-06-30 11:01:00,8.15,17949.0,United Kingdom,5,4401.00,decile01
160544,550461,48185,DOORMAT FAIRY CAKE,670,2011-04-18 13:20:00,6.35,15749.0,United Kingdom,5,4254.50,decile01
291249,562439,84879,ASSORTED COLOUR BIRD ORNAMENT,2880,2011-08-04 18:06:00,1.45,12931.0,United Kingdom,5,4176.00,decile01
422351,573077,M,Manual,1,2011-10-27 14:13:00,4161.06,12536.0,France,5,4161.06,decile01
422376,573080,M,Manual,1,2011-10-27 14:20:00,4161.06,12536.0,France,5,4161.06,decile01
52134,540689,22470,HEART OF WICKER LARGE,1284,2011-01-11 08:43:00,3.21,17450.0,United Kingdom,5,4121.64,decile01
406406,571751,M,Manual,1,2011-10-19 11:18:00,3949.32,12744.0,Singapore,5,3949.32,decile01
533812,581115,22413,METAL SIGN TAKE IT OR LEAVE IT,1404,2011-12-07 12:20:00,2.75,15195.0,United Kingdom,5,3861.00,decile01
16438,537659,21623,VINTAGE UNION JACK MEMOBOARD,600,2010-12-07 16:43:00,6.38,18102.0,United Kingdom,5,3828.00,decile01


In [0]:
dfD['Deli']=0
labels=["decile01","decile02","decile03","decile04","decile05","decile06","decile07","decile08","decile09"]

for i in range(1,10):
  dfD['Deli'][(i-1)*decilN:i*decilN]=labels[i-1]

dfD['Deli'][9*decilN:]="decile10"
dfD

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice,Deli
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,5,168469.60,decile01
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,5,77183.60,decile01
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,5,38970.00,decile01
173382,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,5,8142.75,decile01
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,5,7144.72,decile01
...,...,...,...,...,...,...,...,...,...,...,...
279324,561284,22167,OVAL WALL MIRROR DIAMANTE,1,2011-07-26 12:24:00,0.00,16818.0,United Kingdom,5,0.00,decile10
285657,561916,M,Manual,1,2011-08-01 11:44:00,0.00,15581.0,United Kingdom,5,0.00,decile10
395529,571035,M,Manual,1,2011-10-13 12:50:00,0.00,12446.0,RSA,5,0.00,decile10
439361,574469,22385,JUMBO BAG SPACEBOY DESIGN,12,2011-11-04 11:55:00,0.00,12431.0,Australia,5,0.00,decile10


In [0]:
#各グループのTotalPriceの最小値を取りたい
print(dfD.groupby('Deli').count())
print(39792*9+39796)

          InvoiceNo  StockCode  Description  Quantity  InvoiceDate  UnitPrice  \
Deli                                                                            
decile01      39792      39792        39792     39792        39792      39792   
decile02      39792      39792        39792     39792        39792      39792   
decile03      39792      39792        39792     39792        39792      39792   
decile04      39792      39792        39792     39792        39792      39792   
decile05      39792      39792        39792     39792        39792      39792   
decile06      39792      39792        39792     39792        39792      39792   
decile07      39792      39792        39792     39792        39792      39792   
decile08      39792      39792        39792     39792        39792      39792   
decile09      39792      39792        39792     39792        39792      39792   
decile10      39796      39796        39796     39796        39796      39796   

          CustomerID  Count

In [0]:
#各グループのTotalPriceの最小値を取りたい
dfD.groupby('Deli').min()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
Deli,,,,,,,,,
decile01,536367,50'S CHRISTMAS GIFT BAG LARGE,1,2010-12-01 08:34:00,0.04,12346.0,Australia,5,35.40
decile02,536365,4 PURPLE FLOCK DINNER CANDLES,1,2010-12-01 08:26:00,0.06,12347.0,Australia,5,20.80
decile03,536365,50'S CHRISTMAS GIFT BAG LARGE,1,2010-12-01 08:26:00,0.04,12347.0,Australia,5,17.40
decile04,536365,4 PURPLE FLOCK DINNER CANDLES,1,2010-12-01 08:26:00,0.06,12347.0,Australia,5,15.00
decile05,536367,50'S CHRISTMAS GIFT BAG LARGE,1,2010-12-01 08:34:00,0.06,12347.0,Australia,5,11.80
decile06,536366,4 PURPLE FLOCK DINNER CANDLES,1,2010-12-01 08:28:00,0.06,12347.0,Australia,5,8.50
decile07,536373,4 PURPLE FLOCK DINNER CANDLES,1,2010-12-01 09:02:00,0.04,12347.0,Australia,5,5.80
decile08,536381,4 PURPLE FLOCK DINNER CANDLES,1,2010-12-01 09:41:00,0.04,12347.0,Australia,5,3.75
decile09,536381,4 PURPLE FLOCK DINNER CANDLES,1,2010-12-01 09:41:00,0.06,12362.0,Australia,5,1.95


In [0]:
#それぞれのグループでの合計購買金額の範囲と、それぞれの金額合計値を算出
dfD.groupby('Deli').max()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
Deli,,,,,,,,,
decile01,581586,ZINC WIRE SWEETHEART LETTER TRAY,80995,2011-12-09 12:49:00,8142.75,18287.0,Unspecified,5,168469.60
decile02,581587,ZINC WIRE SWEETHEART LETTER TRAY,400,2011-12-09 12:50:00,35.00,18287.0,Unspecified,5,35.40
decile03,581587,ZINC WILLIE WINKIE CANDLE STICK,500,2011-12-09 12:50:00,20.00,18287.0,Unspecified,5,20.80
decile04,581587,ZINC WIRE KITCHEN ORGANISER,288,2011-12-09 12:50:00,16.95,18287.0,Unspecified,5,17.40
decile05,581587,ZINC WIRE SWEETHEART LETTER TRAY,240,2011-12-09 12:50:00,15.00,18287.0,Unspecified,5,15.00
decile06,581587,ZINC WIRE SWEETHEART LETTER TRAY,160,2011-12-09 12:50:00,11.63,18287.0,Unspecified,5,11.80
decile07,581585,ZINC WIRE SWEETHEART LETTER TRAY,200,2011-12-09 12:31:00,8.50,18287.0,Unspecified,5,8.50
decile08,581585,ZINC WIRE SWEETHEART LETTER TRAY,100,2011-12-09 12:31:00,5.79,18287.0,Unspecified,5,5.80
decile09,581580,ZINC WIRE SWEETHEART LETTER TRAY,50,2011-12-09 12:20:00,3.75,18283.0,Unspecified,5,3.75


In [0]:
#それぞれのグループでの合計購買金額の範囲と、それぞれの金額合計値を算出
dfD.groupby('Deli').sum()

,Quantity,UnitPrice,CustomerID,TotalPrice
Deli,,,,
decile01,2470317,275104.630,595841025.0,4.847556e+06
decile02,556375,148987.760,598601228.0,1.081276e+06
decile03,417499,127286.150,595178324.0,7.594822e+05
decile04,313193,155079.750,597439549.0,6.414016e+05
decile05,369538,132532.690,600635622.0,5.519562e+05
decile06,461897,86272.040,606205704.0,4.033213e+05
decile07,225885,117988.230,617781014.0,2.868994e+05
decile08,193534,87144.270,619680507.0,1.819585e+05
decile09,99805,70435.180,626511925.0,1.095244e+05


In [0]:
ATP=dfD['TotalPrice'].sum()
ATP

8911407.904000001

In [0]:
dfDs=dfD.groupby('Deli').sum()
dfDs

,Quantity,UnitPrice,CustomerID,TotalPrice
Deli,,,,
decile01,2470317,275104.630,595841025.0,4.847556e+06
decile02,556375,148987.760,598601228.0,1.081276e+06
decile03,417499,127286.150,595178324.0,7.594822e+05
decile04,313193,155079.750,597439549.0,6.414016e+05
decile05,369538,132532.690,600635622.0,5.519562e+05
decile06,461897,86272.040,606205704.0,4.033213e+05
decile07,225885,117988.230,617781014.0,2.868994e+05
decile08,193534,87144.270,619680507.0,1.819585e+05
decile09,99805,70435.180,626511925.0,1.095244e+05


In [0]:
#それぞれのグループが売上の何割を占めるか（Pandas.Series形式で売上が小さいグループが先頭、大きいほど後になるようにしてください）
dfDs2 = dfDs.sort_values(by=["TotalPrice"], ascending=True) #total sulfur dioxideについて昇順(小さい順)で並び替え
dfDs2

,Quantity,UnitPrice,CustomerID,TotalPrice
Deli,,,,
decile10,73653,39169.914,628100172.0,4.803261e+04
decile09,99805,70435.180,626511925.0,1.095244e+05
decile08,193534,87144.270,619680507.0,1.819585e+05
decile07,225885,117988.230,617781014.0,2.868994e+05
decile06,461897,86272.040,606205704.0,4.033213e+05
decile05,369538,132532.690,600635622.0,5.519562e+05
decile04,313193,155079.750,597439549.0,6.414016e+05
decile03,417499,127286.150,595178324.0,7.594822e+05
decile02,556375,148987.760,598601228.0,1.081276e+06


In [0]:
dfDs2['share']=0.00
for i in range(10):
  dfDs2['share'][i]=dfDs2['TotalPrice'][i]/ATP*100

dfDs2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Quantity,UnitPrice,CustomerID,TotalPrice,share
Deli,,,,,
decile10,73653,39169.914,628100172.0,4.803261e+04,0.539001
decile09,99805,70435.180,626511925.0,1.095244e+05,1.229035
decile08,193534,87144.270,619680507.0,1.819585e+05,2.041860
decile07,225885,117988.230,617781014.0,2.868994e+05,3.219462
decile06,461897,86272.040,606205704.0,4.033213e+05,4.525899
decile05,369538,132532.690,600635622.0,5.519562e+05,6.193816
decile04,313193,155079.750,597439549.0,6.414016e+05,7.197534
decile03,417499,127286.150,595178324.0,7.594822e+05,8.522584
decile02,556375,148987.760,598601228.0,1.081276e+06,12.133614


In [0]:
dfDs2['share']

Deli
decile10     0.539001
decile09     1.229035
decile08     2.041860
decile07     3.219462
decile06     4.525899
decile05     6.193816
decile04     7.197534
decile03     8.522584
decile02    12.133614
decile01    54.397195
Name: share, dtype: float64

In [0]:
#それぞれのグループが売上の何割を占めるか（Pandas.Series形式で売上が小さいグループが先頭、大きいほど後になるようにしてください）
dfDs2['TotalPrice']

Deli
decile10    4.803261e+04
decile09    1.095244e+05
decile08    1.819585e+05
decile07    2.868994e+05
decile06    4.033213e+05
decile05    5.519562e+05
decile04    6.414016e+05
decile03    7.594822e+05
decile02    1.081276e+06
decile01    4.847556e+06
Name: TotalPrice, dtype: float64

In [0]:
online_retail_data_table

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,5
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,5
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,5
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,5


In [0]:
online_retail_data_table.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
cancel_flg          0
dtype: int64

In [0]:
target_online_retail_data_tb

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,5,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,5,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60


In [0]:
#カラムがCustomerIDと合計金額のあるテーブルを作成してください。
df2 = target_online_retail_data_tb_sort[['CustomerID','TotalPrice']]
df2

,CustomerID,TotalPrice
540421,16446.0,168469.60
61619,12346.0,77183.60
222680,15098.0,38970.00
173382,16029.0,8142.75
348325,17450.0,7144.72
...,...,...
279324,16818.0,0.00
285657,15581.0,0.00
395529,12446.0,0.00
439361,12431.0,0.00


In [0]:
#購買者を10等分にグループ分けします（例：100人いたら、10人ずつにグループ分けします。）。それぞれのグループでの合計購買金額の範囲と、それぞれの金額合計値を算出してください
#1.Desiというカラムを作り、グループ分けしてその名前を入れる
df2['Decile'] = pd.qcut(df2['TotalPrice'], 10, labels=["decile1","decile2","decile3","decile4","decile5","decile6","decile7","decile8","decile9","decile10"]) #quantileを使うとどうなるか？
#2．グループごとの人数を確認する
df2D=df2.groupby('Decile').count()
df2D
#3．全体の人数を確認する
#df2D.sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,CustomerID,TotalPrice
Decile,,
decile1,40495,40495
decile2,44004,44004
decile3,35434,35434
decile4,40025,40025
decile5,39154,39154
decile6,47135,47135
decile7,34302,34302
decile8,40995,40995
decile9,37612,37612


In [0]:
df2sum=df2.groupby('Decile').sum()
df2sum

,CustomerID,TotalPrice
Decile,,
decile1,639310149.0,4.939566e+04
decile2,692892758.0,1.265776e+05
decile3,550737314.0,1.667496e+05
decile4,621169555.0,2.903730e+05
decile5,596505469.0,3.983867e+05
decile6,711099500.0,6.625748e+05
decile7,514376536.0,5.638540e+05
decile8,613583019.0,7.913080e+05
decile9,565759733.0,1.050882e+06


In [0]:
#それぞれのグループでの合計購買金額の範囲と、それぞれの金額合計値を算出してください
#2．グループごとの人数を確認する
df2D=df2.groupby('Decile').count()

540421   NaN
61619    NaN
222680   NaN
173382   NaN
348325   NaN
          ..
279324   NaN
285657   NaN
395529   NaN
439361   NaN
47070    NaN
Name: Desi, Length: 397924, dtype: float64

In [0]:
desi=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
df2['Desi'] = df2['TotalPrice'].quantile(desi)
df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CustomerID,TotalPrice,Desi
540421,16446.0,168469.60,NaN
61619,12346.0,77183.60,NaN
222680,15098.0,38970.00,NaN
173382,16029.0,8142.75,NaN
348325,17450.0,7144.72,NaN
...,...,...,...
279324,16818.0,0.00,NaN
285657,15581.0,0.00,NaN
395529,12446.0,0.00,NaN
439361,12431.0,0.00,NaN


In [0]:
srD2[0.1]

1.95

In [0]:
# working place. everything 
def homework(target_online_retail_data_tb):
    !!WRITE ME!!
    return my_result  